## <center> <font color='purple'> Dynamic Ensemble Machine Learning Models (Mix Pool)</font></center> 
#### <center>Firuz Juraev (Sungkyunkwan Unniversity)</center>

### <font color='green'> Libraries 

#### <font color='blue'> Basic Libraries 

In [1]:
import pandas as pd
import numpy as np 
from numpy import mean
from numpy import std
import seaborn as sns 
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### <font color='blue'> Single ML Models Libraries 

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

#### <font color='blue'> Static ML Models Libraries 

In [3]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#### <font color='blue'> DES Libraries 

In [4]:
from deslib.des import DESP
from deslib.des import KNORAE
from deslib.des import KNORAU
from deslib.des import METADES
from deslib.des import DESKNN # new 
from deslib.des import KNOP # new 

#### <font color='blue'> DCS Libraries 

In [5]:
from deslib.dcs import MCB

#### <font color='blue'> Processing Libraries 

In [6]:
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

from sklearn.metrics import (accuracy_score,
                             precision_score,
                             recall_score, 
                             f1_score,
                             roc_auc_score, 
                             auc)
from sklearn.metrics import roc_curve, roc_auc_score 
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#### <font color='blue'>Libraries for Critical Diagram 

In [7]:
from sklearn.model_selection import StratifiedKFold
from scipy.stats import wilcoxon, friedmanchisquare, rankdata
from Orange.evaluation import compute_CD, graph_ranks
from sklearn.metrics import mean_absolute_error

### <font color='green'> Load Dataset 

In [8]:
train_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_train_data_4.csv")
test_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_test_data_4.csv")

X_train = train_dataset_upsampled.drop(["DEAD"], axis=1) 
y_train = train_dataset_upsampled["DEAD"]

X_test = test_dataset_upsampled.drop(["DEAD"], axis=1) 
y_test = test_dataset_upsampled["DEAD"]

In [9]:
columns = X_train.columns.to_list()

### <font color='green'> Data Normalization

In [10]:
Min_max_scaler = MinMaxScaler().fit(X_train)

## Scaling 
X_train_mm_scaled = Min_max_scaler.transform(X_train)
X_test_mm_scaled = Min_max_scaler.transform(X_test)

## Numpy Array to DataFrame 
df_train_mm_scaled = pd.DataFrame(X_train_mm_scaled, columns = columns)
df_test_mm_scaled = pd.DataFrame(X_test_mm_scaled, columns = columns)

### <font color='green'> Feature Selection

In [11]:
tain_mm_scaled_df = df_train_mm_scaled[:]
tain_mm_scaled_df["DEAD"] = y_train 

test_mm_scaled_df = df_test_mm_scaled[:] 
test_mm_scaled_df["DEAD"] = y_test


importances = mutual_info_classif(df_train_mm_scaled, y_train)
feat_importance = pd.Series(importances, tain_mm_scaled_df.columns[0:len(tain_mm_scaled_df.columns)-1])
    
feat_importance = feat_importance.sort_values(ascending=False)
    
selected_features = feat_importance[:30]
selected_features_list_mm_scaled = selected_features.index.to_list()


tain_mm_scaled_df[selected_features_list_mm_scaled].head(2)

,temperature_mean,respRate_std,respRate_var,skinTemperature_std,skinTemperature_var,heartRate_std,heartRate_var,bpCuffMean_var,sao2_var,sao2_std,...,BIRTH_WEIGHT,bpCuffSystolic_mean,bpCuffDiastolic_mean,temperature_var,temperature_std,glucometer_mean,sao2_mean,bpCuffMean_mean,PLATELET,D10W_MEAN
0,0.775318,0.422245,0.178291,0.134078,0.017977,0.22911,0.057937,0.053294,0.009568,0.097817,...,0.264267,0.602649,0.457143,0.015695,0.125281,0.312593,0.881455,0.310696,0.281330,0.151138
1,0.709668,0.583964,0.341014,0.086672,0.007512,0.28258,0.086102,0.020317,0.006551,0.080938,...,0.437819,0.644907,0.479330,0.003770,0.061399,0.272551,0.926056,0.351928,0.392157,0.196784


In [12]:
X_train_mm = df_train_mm_scaled[selected_features_list_mm_scaled][:]
X_test_mm = df_test_mm_scaled[selected_features_list_mm_scaled][:]

### <font color='purple'> Hold-out Test (With Mix ML) - (+FS, +HO)

In [13]:
def get_individual_result(model, dsel_x, dsel_y, test_x, test_y): 
    model.fit(dsel_x, dsel_y)
    y_preds = model.predict(test_x) 
    yproba = model.predict_proba(test_x)[::,1] 
    
    acc = accuracy_score(test_y, y_preds)
    prec = precision_score(test_y, y_preds)
    rec = recall_score(test_y, y_preds)
    f1 = f1_score(test_y, y_preds)
    fpr, tpr, _ = roc_curve(test_y,  yproba) 
    auc = roc_auc_score(test_y, yproba)
    
    return {"acc": acc, "prec": prec, "rec": rec, "f1":f1, "fpr": fpr, "tpr":tpr, "auc": auc}
    

In [14]:
def hold_out_mix_ML(): 
    rng = np.random.RandomState(42) 
    X_train, X_dsel, y_train_en, y_dsel = train_test_split(X_train_mm, y_train, test_size=0.40, random_state=rng)
    
    model_svc = SVC(kernel='linear', C=0.007, gamma=0.2, degree=3, probability=True, class_weight='balanced')
#     model_dt = DecisionTreeClassifier(criterion='entropy', max_depth=3) # depth was 3  
    model_lr = LogisticRegression(penalty='l2', C=0.002)
    model_ml_perceptron = MLPClassifier(solver='adam', max_iter=11, verbose=10,learning_rate_init=.003)
    
    model_rf  = RandomForestClassifier(criterion='gini', n_estimators=100, max_depth=3)
    model_cat = MLPClassifier(solver='adam', max_iter=11, verbose=10,learning_rate_init=.003)
    model_lgb = LGBMClassifier(max_depth=1, n_estimators=150, objective="binary")
    model_ada = AdaBoostClassifier(n_estimators=250, learning_rate=0.01)
    
    model_svc.fit(X_train, y_train_en)
#     model_dt.fit(X_train, y_train_en)
    model_lr.fit(X_train, y_train_en)
    model_ml_perceptron.fit(X_train, y_train_en)
    model_rf.fit(X_train, y_train_en) 
    model_cat.fit(X_train, y_train_en) 
    model_lgb.fit(X_train, y_train_en)
    model_ada.fit(X_train, y_train_en)
    
    classifiers_names = ["MLP", "Logistic Regression", "SVC", "Random Forest", 
                         "CatBoost", "AdaBoost"] # "Decision Tree",
    # "AdaBoost" , "LGBM"
    pool_classifiers = [model_ml_perceptron, 
                        model_lr, 
                        model_svc, 
                        model_rf, 
                        model_cat,
                        model_ada] 
    c_acc_list = [] 
    
    for cls in pool_classifiers:
        y_preds_c = cls.predict(X_test_mm) 
        c_acc_list.append(accuracy_score(y_test, y_preds_c))           
        
        
    classifiers_results =  {'name': classifiers_names, 
                            'accuracy': c_acc_list}
    
    clsDF = pd.DataFrame.from_dict(classifiers_results)
        
    # DES STARTS
    
    knorau = KNORAU(pool_classifiers)
    kne = KNORAE(pool_classifiers)  
    metades = METADES(pool_classifiers)
    desknn = DESKNN(pool_classifiers)
    mcb = MCB(pool_classifiers)
    desp = DESP(pool_classifiers)
    knop = KNOP(pool_classifiers)

    fire_knorau = KNORAU(pool_classifiers, DFP=True, k=7) 
    fire_kne = KNORAE(pool_classifiers, DFP=True, k=9)
    fire_metades = METADES(pool_classifiers, DFP=True, k=9) 
    fire_desknn = DESKNN(pool_classifiers, DFP=True, k=9)
    fire_mcb = MCB(pool_classifiers, DFP=True, k=7) # 7 was 96%
    fire_desp = DESP(pool_classifiers, DFP=True, k=9)
    fire_knop = KNOP(pool_classifiers, DFP=True, k=15)
    
    ensemble_classifiers = [fire_knorau, knorau, fire_kne, kne, fire_metades, metades, fire_desknn, desknn, 
                            fire_mcb, mcb, fire_desp, desp, fire_knop, knop]
    
    ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                      "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]
    
    acc_list = [] 
    precision_list = [] 
    recall_list = []
    f1_lists = [] 
    auc_list = [] 
    fpr_list = [] 
    tpr_list = []
    
    for e_cls in ensemble_classifiers:
        results_dct = get_individual_result(e_cls, X_dsel, y_dsel, X_test_mm, y_test)
        acc_list.append(results_dct["acc"])
        precision_list.append(results_dct["prec"])
        recall_list.append(results_dct["rec"]) 
        f1_lists.append(results_dct["f1"]) 
        auc_list.append(results_dct["auc"])
        fpr_list.append(results_dct["fpr"])
        tpr_list.append(results_dct["tpr"])

    
    results =  {'name': ensemble_names, 
                'accuracy': acc_list, 
                'precision': precision_list, 
                'recall': recall_list, 'f1': f1_lists, 
                'auc': auc_list, 
                'tpr': tpr_list, 'fpr': fpr_list}
    
    df = pd.DataFrame.from_dict(results)
    
    return df, clsDF

In [15]:
results_data = []
classifier_results_data = []
for i in range(0, 15):
    result, cls_results = hold_out_mix_ML()
    results_data.append(result)
    classifier_results_data.append(cls_results)
    

fireResultsDF = pd.concat(results_data)
classifiersResultsDF = pd.concat(classifier_results_data)

Iteration 1, loss = 0.62349346
Iteration 2, loss = 0.49625272
Iteration 3, loss = 0.41491680
Iteration 4, loss = 0.36994177
Iteration 5, loss = 0.34294970
Iteration 6, loss = 0.32854334
Iteration 7, loss = 0.31759707
Iteration 8, loss = 0.29883632
Iteration 9, loss = 0.29361184
Iteration 10, loss = 0.28752699
Iteration 11, loss = 0.28084111
Iteration 1, loss = 0.65465857
Iteration 2, loss = 0.55144608
Iteration 3, loss = 0.46328015
Iteration 4, loss = 0.40607787
Iteration 5, loss = 0.36860477
Iteration 6, loss = 0.34776775
Iteration 7, loss = 0.33021908
Iteration 8, loss = 0.31517893
Iteration 9, loss = 0.30116474
Iteration 10, loss = 0.29208819
Iteration 11, loss = 0.28745310
Iteration 1, loss = 0.61827501
Iteration 2, loss = 0.49948216
Iteration 3, loss = 0.41593341
Iteration 4, loss = 0.37188559
Iteration 5, loss = 0.34601433
Iteration 6, loss = 0.32754257
Iteration 7, loss = 0.31025484
Iteration 8, loss = 0.30467338
Iteration 9, loss = 0.28983019
Iteration 10, loss = 0.28041367
Ite

Iteration 1, loss = 0.61355468
Iteration 2, loss = 0.50032286
Iteration 3, loss = 0.42538030
Iteration 4, loss = 0.38172026
Iteration 5, loss = 0.35406845
Iteration 6, loss = 0.33520212
Iteration 7, loss = 0.32109391
Iteration 8, loss = 0.31160903
Iteration 9, loss = 0.30237891
Iteration 10, loss = 0.29044512
Iteration 11, loss = 0.28657605
Iteration 1, loss = 0.62401483
Iteration 2, loss = 0.51074527
Iteration 3, loss = 0.42570557
Iteration 4, loss = 0.37743687
Iteration 5, loss = 0.34758385
Iteration 6, loss = 0.32658745
Iteration 7, loss = 0.31085331
Iteration 8, loss = 0.30076683
Iteration 9, loss = 0.29013300
Iteration 10, loss = 0.28122871
Iteration 11, loss = 0.27362090
Iteration 1, loss = 0.63008769
Iteration 2, loss = 0.51607474
Iteration 3, loss = 0.43184912
Iteration 4, loss = 0.38518370
Iteration 5, loss = 0.35512210
Iteration 6, loss = 0.33263438
Iteration 7, loss = 0.31637299
Iteration 8, loss = 0.30363817
Iteration 9, loss = 0.29419987
Iteration 10, loss = 0.28396794
Ite

In [16]:
ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                  "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]

accuracy = [] 
accuracy_std =  [] 
precision = [] 
precision_std = [] 
recall = [] 
recall_std = [] 
f1_score = [] 
f1_std = []
auc_score = []
auc_std = []


for n in ensemble_names:
    accuracy.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.mean(), 3))
    accuracy_std.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.std(), 3))
    precision.append(round(fireResultsDF[fireResultsDF.name == n].precision.mean(), 3))
    precision_std.append(round(fireResultsDF[fireResultsDF.name == n].precision.std(), 3))
    recall.append(round(fireResultsDF[fireResultsDF.name == n].recall.mean(), 3))
    recall_std.append(round(fireResultsDF[fireResultsDF.name == n].recall.std(), 3))
    f1_score.append(round(fireResultsDF[fireResultsDF.name == n].f1.mean(), 3))
    f1_std.append(round(fireResultsDF[fireResultsDF.name == n].f1.std(), 3))
    auc_score.append(round(fireResultsDF[fireResultsDF.name == n].auc.mean(), 3))
    auc_std.append(round(fireResultsDF[fireResultsDF.name == n].auc.std(), 3))
    
final_results = {"method": ensemble_names, 
                     "accuracy": accuracy, 
                     "accuracy_std": accuracy_std,
                     "precision": precision, 
                     "precision_std": precision_std,
                     "recall": recall, 
                     "recall_std": recall_std,
                     "f1_score": f1_score, 
                     "f1_std": f1_std, 
                     "auc": auc_score, 
                     "auc_std": auc_std}

finalResultsDF = pd.DataFrame.from_dict(final_results)

In [22]:
finalResultsDF

,method,accuracy,accuracy_std,precision,precision_std,recall,recall_std,f1_score,f1_std,auc,auc_std
0,FIRE-KNORA-U,0.963,0.002,0.931,0.004,1.000,0.000,0.964,0.002,0.984,0.001
1,KNORA-U,0.962,0.003,0.929,0.005,1.000,0.000,0.963,0.003,0.984,0.001
2,FIRE-KNORA-E,0.968,0.002,0.939,0.003,1.000,0.000,0.969,0.002,0.986,0.001
3,KNORA-E,0.968,0.002,0.939,0.003,1.000,0.000,0.969,0.002,0.986,0.001
4,FIRE-METADES,0.969,0.002,0.941,0.004,1.000,0.000,0.970,0.002,0.988,0.001
5,METADES,0.968,0.002,0.940,0.004,1.000,0.000,0.969,0.002,0.988,0.001
6,FIRE-DESKNN,0.943,0.002,0.951,0.003,0.935,0.000,0.943,0.002,0.985,0.001
7,DESKNN,0.944,0.001,0.951,0.003,0.935,0.000,0.943,0.001,0.985,0.001
8,FIRE-MCB,0.955,0.003,0.935,0.003,0.979,0.006,0.956,0.003,0.985,0.001
9,MCB,0.957,0.003,0.936,0.005,0.982,0.004,0.959,0.003,0.985,0.001


In [23]:
# finalResultsDF.to_csv("Results/mix_pool_results/des_mix_pool_6_cls_results_3-3.csv", index=False)

In [19]:
classifiers_names = ["MLP", "Logistic Regression", "SVC", "Random Forest", "CatBoost", "AdaBoost"] 
# "AdaBoost", "LGBM"

accuracy_list = []
accuracy_std_list = []
for n in classifiers_names:
    accuracy_list.append(classifiersResultsDF[classifiersResultsDF.name == n].accuracy.mean())
    accuracy_std_list.append(classifiersResultsDF[classifiersResultsDF.name == n].accuracy.std())

final_cls_results = {"classifier": classifiers_names, 
                     "accuracy": accuracy_list, 
                     "accuracy_std": accuracy_std_list}

finalClassifierResultsDF = pd.DataFrame.from_dict(final_cls_results)

In [20]:
finalClassifierResultsDF

,classifier,accuracy,accuracy_std
0,MLP,0.947465,1.244374e-02
1,Logistic Regression,0.941748,4.596760e-16
2,SVC,0.932578,0.000000e+00
3,Random Forest,0.938583,1.728244e-03
4,CatBoost,0.942107,1.376449e-02
5,AdaBoost,0.950378,2.298380e-16


In [21]:
finalResultsDF.describe().T

,count,mean,std,min,25%,50%,75%,max
accuracy,14.0,0.960929,8.570650e-03,0.943,0.95750,0.9635,0.96775,0.969
accuracy_std,14.0,0.003786,4.245877e-03,0.001,0.00200,0.0020,0.00300,0.017
precision,14.0,0.938857,6.443329e-03,0.929,0.93525,0.9390,0.94100,0.951
precision_std,14.0,0.003929,8.287419e-04,0.003,0.00300,0.0040,0.00475,0.005
recall,14.0,0.986071,2.320300e-02,0.935,0.97975,1.0000,1.00000,1.000
recall_std,14.0,0.004214,9.266802e-03,0.000,0.00000,0.0000,0.00300,0.032
f1_score,14.0,0.961786,9.014335e-03,0.943,0.95925,0.9645,0.96875,0.970
f1_std,14.0,0.003786,4.245877e-03,0.001,0.00200,0.0020,0.00300,0.017
auc,14.0,0.985929,1.384768e-03,0.984,0.98500,0.9860,0.98675,0.988
auc_std,14.0,0.001000,4.500519e-19,0.001,0.00100,0.0010,0.00100,0.001
